In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src/')
from CellType_PSY import *

# Structure in Common vs Corr Null Hypo

In [ ]:
def FixSubiculum(DF):
    X = DF.loc["Subiculum_dorsal_part"]
    Y = DF.loc["Subiculum_ventral_part"]
    Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
    DF.loc["Subiculum"] = Z
    DF = DF.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])
    return DF

In [ ]:
ASD_STR_Bias = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col=0)
ASD_STR_Bias = FixSubiculum(ASD_STR_Bias)
#SC_Agg_Bias = pd.read_csv("dat/Bias/ASD.MERFISH.Agg.adj.bias.csv", index_col=0)
SC_Agg_Bias = pd.read_csv("dat/Bias/ASD.MERFISH.AggV2.adj.bias.csv", index_col=0)

In [ ]:
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
ASD_Circuits.append("Subiculum")

In [ ]:
for i,row in ASD_STR_Bias.iterrows():
    ASD_STR_Bias.loc[i, "SC_Bias"] = SC_Agg_Bias.loc[i, "EFFECT"]
    ASD_STR_Bias.loc[i, "SC_Rank"] = int(SC_Agg_Bias.loc[i, "Rank"])
    if i in ASD_Circuits:
        ASD_STR_Bias.loc[i, "isCir"] = 1
    else:
        ASD_STR_Bias.loc[i, "isCir"] = 0

In [ ]:
R,P = pearsonr(ASD_STR_Bias["EFFECT"].values, ASD_STR_Bias["SC_Bias"].values)
print(R, P)

In [ ]:
STR_Com = len(set(ASD_STR_Bias.head(50).index.values).intersection(set(SC_Agg_Bias.head(50).index.values)))
STR_Com

In [ ]:
CompareList(SC_Agg_Bias.head(50).index.values, ASD_Circuits)

In [ ]:
tmp_ASD_STR_Bias = ASD_STR_Bias.copy(deep=True)

In [ ]:
variances = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.2, 0.3]
Corr_dat_at_var = []
STR_in_Comm_at_var = []
for variance in variances:
    for i in range(50):
        noise = np.random.normal(0, np.sqrt(variance), tmp_ASD_STR_Bias['EFFECT'].shape)
        tmp_ASD_STR_Bias["EFFECT2"] = tmp_ASD_STR_Bias["EFFECT"] + noise
        R, p = pearsonr(ASD_STR_Bias["EFFECT"].values, tmp_ASD_STR_Bias["EFFECT"].values)
        tmp_ASD_STR_Bias = tmp_ASD_STR_Bias.sort_values("EFFECT2", ascending=False)
        STR_Com = len(set(ASD_STR_Bias.head(50).index.values).intersection(set(tmp_ASD_STR_Bias.head(50).index.values)))
        Corr_dat_at_var.append(R)
        STR_in_Comm_at_var.append(STR_Com)
Corr_dat_at_var = np.array(Corr_dat_at_var)
STR_in_Comm_at_var = np.array(STR_in_Comm_at_var)

In [ ]:
plt.figure(dpi=300, figsize=(8, 6))
plt.scatter(STR_in_Comm_at_var, Corr_dat_at_var, color='blue', s=100, alpha=0.7, edgecolor='k')
plt.scatter(27, 0.732, color="red", s=100, label='MERSIH-ISH', marker="x")

plt.xlabel("STR in Common")
plt.ylabel("Bias Corr")
plt.legend(fontsize=12)

# Add grid lines
plt.grid(True, linestyle='--', alpha=0.6)

# Try other things that might help

In [ ]:
MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.csv",index_col=0)
MERFISH_Z2_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_VolMean.Z2.csv", index_col=0)
MERFISH_Z2_Cell2 = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv", index_col=0)

In [ ]:
head = MERFISH_Z2_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]

MERFISH_Z2_Cell.columns = new_head
MERFISH_Z2_Vol.columns = new_head
MERFISH_Z2_Cell2.columns = new_head

In [ ]:
ASD_GW = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
#del ASD_GW[23126]

In [ ]:
ISH_SC_CorrDF = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v2.csv", index_col=0)
ASD_GW_adj = {}
for k,v in ASD_GW.items():
    if k in ISH_SC_CorrDF.index.values:
        ASD_GW_adj[k] = v * (ISH_SC_CorrDF.loc[k, "V2_V3_CT_Corr"]**2)

In [ ]:
len(ASD_GW_adj)

In [ ]:
ASD_Bias_MERFISH_Z2_Cell = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW_adj)
ASD_Bias_MERFISH_Z2_Vol = AvgSTRZ_Weighted(MERFISH_Z2_Vol, ASD_GW_adj)
ASD_Bias_MERFISH_Z2_Cell2 = AvgSTRZ_Weighted(MERFISH_Z2_Cell2, ASD_GW_adj)

In [ ]:
#ASD_Bias_MERFISH_Z2_Vol.head(50)

In [ ]:
STR_Com = len(set(ASD_STR_Bias.head(50).index.values).intersection(set(ASD_Bias_MERFISH_Z2_Vol.head(50).index.values)))
STR_Com

In [ ]:
STR_Com = len(set(ASD_STR_Bias.head(50).index.values).intersection(set(ASD_Bias_MERFISH_Z2_Cell.head(50).index.values)))
STR_Com

In [ ]:
STR_Com = len(set(ASD_Bias_MERFISH_Z2_Cell.head(50).index.values).intersection(set(ASD_Bias_MERFISH_Z2_Vol.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_STR_Bias, ASD_Bias_MERFISH_Z2_Cell, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(ASD_STR_Bias, ASD_Bias_MERFISH_Z2_Vol, "ISH", "MERFISH")

In [ ]:
MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv",index_col=0)
head = MERFISH_Z2_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Z2_Cell.columns = new_head
ISH_Z2 = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

In [ ]:
ASD_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW) 
ASD_MERFISH_adj = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW_adj) 

In [ ]:
BiasCorrelation(ASD_STR_Bias, ASD_MERFISH_adj, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(ASD_STR_Bias, ASD_MERFISH_adj, "ISH", "MERFISH")

In [ ]:
ASD_ISH = pd.read_csv("dat/Bias/ASD.ISH.bias.csv", index_col=0)
ASD_MERFISH_adj = pd.read_csv("dat/Bias/ASD.MERFISH.Agg.adj.bias.csv", index_col=0)

In [ ]:
BiasCorrelation(ASD_ISH, ASD_MERFISH_adj, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(SC_Agg_Bias, ASD_MERFISH_adj, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(SC_Agg_Bias, ASD_Bias_MERFISH_Z2_Cell, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(SC_Agg_Bias, ASD_Bias_MERFISH_Z2_Cell2, "ISH", "MERFISH")

In [ ]:
BiasCorrelation(SC_Agg_Bias, ASD_Bias_MERFISH_Z2_Vol, "ISH", "MERFISH")

In [ ]:
Bias1 = SC_Agg_Bias
Bias2 = ASD_MERFISH_adj
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_MERFISH_adj
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = SC_Agg_Bias
Bias2 = ASD_ISH
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_STR_Bias
Bias2 = ASD_Bias_MERFISH_Z2_Vol
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_Z2_Vol.head(50).index.values))

In [ ]:
Bias1 = ASD_Bias_MERFISH_Z2_Cell
Bias2 = ASD_STR_Bias
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com